In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [ ]:
%run -p ../train_cnf_disentangle_rl_semisup.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/infocnf_cond_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_6_nsup4k_beta_0_001_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 6.0 --labeled_batch_size 450 --num_train_sup 4000 --beta 0.001
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_semisup.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWr

Load semi-sup datasets for CIFAR10


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0001 | Time 98.3373, Epoch Time 1560.0854(1560.0854), Bit/dim 7.0173(best: inf), Bit/dim Marginal 7.0170(best: inf), Xent 2.0832, Loss 8.0589, Error 0.7076(best: inf)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0110 | Time 13.5878(13.7929) | Bit/dim 6.9789(7.3713) | Xent 2.0808(2.0931) | Loss 18.1512(19.3632) | Error 0.6956(0.7078) Steps 544(522.35) | Grad Norm 8.7327(6.0049) | Total Time 0.00(0.00)
Iter 0120 | Time 13.2595(13.7336) | Bit/dim 6.9533(7.2658) | Xent 2.0297(2.0841) | Loss 17.7941(18.9701) | Error 0.6867(0.7050) Steps 538(526.75) | Grad Norm 3.5179(6.0689) | Total Time 0.00(0.00)
Iter 0130 | Time 14.3717(13.8048) | Bit/dim 6.9220(7.1794) | Xent 2.0018(2.0728) | Loss 17.7687(18.6701) | Error 0.6711(0.7005) Steps 544(533.93) | Grad Norm 4.4384(6.6068) | Total Time 0.00(0.00)
Iter 0140 | Time 14.2905(13.7746) | Bit/dim 6.8954(7.1065) | Xent 1.9783(2.0604) | Loss 17.7025(18.4345) | Error 0.6378(0.6932) Steps 574(538.01) | Grad Norm 4.1636(6.7687) | Total 

...

Epoch 0009 | Time 93.6513, Epoch Time 1688.5339(1572.3077), Bit/dim 4.4714(best: 4.5667), Bit/dim Marginal 4.4375(best: 4.5267), Xent 1.8440, Loss 5.3934, Error 0.6512(best: 0.6401)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0920 | Time 14.3697(15.0435) | Bit/dim 4.3962(4.4482) | Xent 1.3076(1.3461) | Loss 11.7214(12.6794) | Error 0.4644(0.4721) Steps 574(595.06) | Grad Norm 25.6140(22.8139) | Total Time 0.00(0.00)
Iter 0930 | Time 15.0649(14.9399) | Bit/dim 4.4212(4.4370) | Xent 1.2814(1.3407) | Loss 11.8083(12.4358) | Error 0.4444(0.4694) Steps 604(590.64) | Grad Norm 14.0110(21.3531) | Total Time 0.00(0.00)
Iter 0940 | Time 14.8105(14.9776) | Bit/dim 4.3770(4.4233) | Xent 1.2221(1.3232) | Loss 11.8024(12.2491) | Error 0.4200(0.4659) Steps 598(591.13) | Grad Norm 14.7769(19.6575) | Total Time 0.00(0.00)
Iter 0950 | Time 14.7761(14.9801) | Bit/dim 4.3646(4.4065) | Xent 1.1438(1.3071) | Loss 11.5931(12.0872) | Error 0.4156(0.4611) Steps 598(594.92) | Grad Norm 11.3419(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0010 | Time 86.6552, Epoch Time 1688.2605(1575.7862), Bit/dim 4.3064(best: 4.4714), Bit/dim Marginal 4.2804(best: 4.4375), Xent 1.6923, Loss 5.1525, Error 0.5963(best: 0.6401)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 1030 | Time 15.0349(14.9623) | Bit/dim 4.3127(4.3360) | Xent 1.1612(1.2489) | Loss 11.3357(12.1557) | Error 0.3956(0.4441) Steps 550(591.55) | Grad Norm 10.3908(15.3535) | Total Time 0.00(0.00)
Iter 1040 | Time 14.8749(14.9795) | Bit/dim 4.2786(4.3196) | Xent 1.3372(1.2477) | Loss 11.4359(11.9663) | Error 0.4644(0.4439) Steps 556(588.50) | Grad Norm 30.7584(15.9803) | Total Time 0.00(0.00)
Iter 1050 | Time 14.8721(15.0361) | Bit/dim 4.2902(4.3153) | Xent 1.2742(1.2396) | Loss 11.5425(11.8596) | Error 0.4533(0.4401) Steps 574(592.76) | Grad Norm 18.9550(18.0330) | Total Time 0.00(0.00)
Iter 1060 | Time 15.0392(15.0385) | Bit/dim 4.2793(4.3051) | Xent 1.1606(1.2343) | Loss 11.4985(11.7555) | Error 0.4089(0.4392) Steps 634(593.48) | Grad Norm 9.1401(1

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0011 | Time 82.8374, Epoch Time 1719.3500(1580.0932), Bit/dim 4.2174(best: 4.3064), Bit/dim Marginal 4.1975(best: 4.2804), Xent 1.6326, Loss 5.0337, Error 0.5762(best: 0.5963)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 1130 | Time 15.5043(15.3202) | Bit/dim 4.1785(4.2336) | Xent 1.1740(1.1735) | Loss 11.0134(11.8833) | Error 0.3933(0.4180) Steps 538(597.45) | Grad Norm 20.3663(15.4776) | Total Time 0.00(0.00)
Iter 1140 | Time 15.9599(15.3964) | Bit/dim 4.1971(4.2232) | Xent 1.1378(1.1667) | Loss 11.1945(11.7079) | Error 0.3956(0.4167) Steps 568(599.00) | Grad Norm 19.5740(15.7234) | Total Time 0.00(0.00)
Iter 1150 | Time 16.1205(15.4225) | Bit/dim 4.1807(4.2131) | Xent 1.1016(1.1548) | Loss 11.1218(11.5509) | Error 0.4000(0.4105) Steps 610(598.26) | Grad Norm 12.3763(15.6350) | Total Time 0.00(0.00)
Iter 1160 | Time 14.9888(15.4610) | Bit/dim 4.1993(4.2078) | Xent 1.0827(1.1546) | Loss 11.2060(11.4696) | Error 0.3911(0.4114) Steps 580(603.06) | Grad Norm 18.4461(